<a href="https://colab.research.google.com/github/31Wilson13/basketball_player_detection/blob/main/detection_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages

In [ ]:
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-1i6fev5e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore /tmp/pip-req-build-1i6fev5e
  Resolved https://github.com/facebookresearch/fvcore to commit 9d683aae73fb899dd35d6cf6720e5ef567761c57
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-4hnmx7q2
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-4hnmx7q2
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.6-py3-none-any.whl size=65324 sha256=09fc84ab2bd522a017478b69bacb34b7b0c7c3405241e7d7e3cb3eb8432bc5e2
  Stored in direc

In [ ]:
# Setup detectron2 logger
import numpy as np
from itertools import compress
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import numpy as np
import pandas as pd
import math
import cv2
import random
from google.colab.patches import cv2_imshow
from shapely.geometry import Point, Polygon


# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
import imutils
from detectron2.utils.visualizer import Visualizer


# load player tracking model - top

In [ ]:
cfg1 = get_cfg()
cfg1.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg1.DATASETS.TRAIN = ("category_train",)
cfg1.DATASETS.TEST = ()
cfg1.DATALOADER.NUM_WORKERS = 2
cfg1.MODEL.ROI_HEADS.NUM_CLASSES = 1
# cfg1.MODEL.DEVICE = 'cpu'


cfg1.MODEL.WEIGHTS = os.path.join('/content/drive/MyDrive/Colab Notebooks/player_tracking/', "myplayermodel_1.pth")
cfg1.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg1.DATASETS.TEST = ("/content/drive/MyDrive/Colab Notebooks/data/test", )
predictor1 = DefaultPredictor(cfg1)


[09/27 02:46:06 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/Colab Notebooks/player_tracking/myplayermodel_1.pth ...


# create sample images

In [ ]:
# import os

# os.chdir('你要使用的影片路徑')  # 使用 Colab 要換路徑使用

from moviepy.editor import *

video = VideoFileClip("/content/drive/MyDrive/Colab Notebooks/2k_top2.mp4")

for i in range(3,25):#因我是固定時間間隔做截圖，因此用loop去跑秒數

    second = i*0.1 #每50秒做一張截圖

    frame = video.save_frame(f"frame%d.jpg"%i, t = second) #讓截圖檔名依照順序

    print('ok')

In [ ]:
img = cv2.imread('frame3.jpg')

uci_color = [255,0,0]
away_color = [0,0,255]

players_output = predictor1(img)
instances = players_output["instances"]
pred_boxes = instances.get("pred_boxes")
pred_classes = instances.get("pred_classes")
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg1.DATASETS.TRAIN[0]), scale=1.0)
v = v.draw_instance_predictions(players_output["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])


# draw players

In [ ]:
imgg = img.copy()
court_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/court3.jpg')

src_pts = np.array([

        [232,155],
         [215,940],
        [1702,940],
         [1688,155]
    ])

cv2.polylines(imgg, [src_pts], isClosed=True, color=[255,0,0], thickness=2)

cv2_imshow(imgg)

dst_pts = np.array([
    [0, 60],          # top left corner
    [0, 1733],      # middle bottom corner
    [2873, 1733],
    [2873, 60],        # top right rorner
    ])

cv2.polylines(court_img, [dst_pts], isClosed=True, color=[255,0,0], thickness=2)

cv2_imshow(court_img)

# draw rim

In [ ]:
img_rim = img.copy()
def drawrim(im, showResult = False):
  color = [0, 255, 0]
  thickness = 20
  radius = 1
  rim_pos = [260, 540]
  cv2.circle(im, rim_pos, radius, color, thickness)
  if showResult:
    cv2_imshow(im)
drawrim(img_rim, True)

In [ ]:
# img = cv2.imread('frame82.jpg')
def homographyTransform(im, img_dst, showResult=False):

  # Calculate Homography
  h, status = cv2.findHomography(src_pts, dst_pts)
  img_out = cv2.warpPerspective(im, h, (img_dst.shape[1], img_dst.shape[0]))

  if showResult:
    cv2_imshow(img_out)

  return img_out

# Try out
img_out = homographyTransform(img_rim, court_img, True)

In [ ]:
# Use the boxes info from the tensor prediction result
#
# x1,y1 ------
# |          |
# |          |
# |          |
# --------x2,y2
#





def drawPlayers(im, pred_boxes, pred_classes, showResult=False):
  color = [255, 0, 0]
  thickness = 20
  radius = 1

  i  = 0
  # print(i)
  for box in pred_boxes:
    # print(i)
    # Include only class Person
    if pred_classes[i] == 0:

      x1 = int(box[0])
      y1 = int(box[1])
      x2 = int(box[2])
      y2 = int(box[3])

      yc = y1 + int((y2 - y1) / 2)
      player_pos = (x2, yc)

      court = Polygon(dst_pts)

      # Draw only players that are within the basketball court
      if Point(player_pos).within(court):
        if showResult:
          print("[% 3d, % 3d]" %(yc, x2))

        cv2.circle(im, player_pos, radius, color, thickness)
      i = i + 1
    else:
      i = i + 1

  if showResult:
    cv2_imshow(im)


drawPlayers(img_out, pred_boxes, pred_classes, True)


In [ ]:
def getRimMask(im):
  lower_range = np.array([0,255,0])                         # Set the Lower range value of blue in BGR
  upper_range = np.array([155,255,155])                     # Set the Upper range value of blue in BGR
  mask = cv2.inRange(im, lower_range, upper_range)     # Create a mask with range
  result = cv2.bitwise_and(im, im, mask = mask)   # Performing bitwise and operation with mask in img variable
  # cv2_imshow(result)

  return cv2.inRange(result, lower_range, upper_range)

# Try out
Rimmask = getRimMask(img_out)
# cv2_imshow(Rimmask)
def mask_rim(rim, showResult = False):
    cnts = cv2.findContours(Rimmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts_width = []
    for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      cnts_width.append(h)
    # print(cnts_width > 8)
    cnts_qualified = [ True if item > 4 else False for item in cnts_width ]
    # res = [ True if item in animal_list else False for item in your_list ]
    cnts_new = []
    for i in range(0,len(cnts)):
      # print(i)
      cnts_new.append(cnts[i][0])
    # cnts_new = [cnts_new[i] for i in (1,2,5)]
    cnts_new = list(compress(cnts_new, cnts_qualified))
    if showResult:
      cv2_imshow(img_out)
      cv2_imshow(Rimmask)
    return cnts_new


cnts_rim = mask_rim(Rimmask, True)
cnts_rim

In [ ]:
def getPlayersMask(im):
  lower_range = np.array([255,0,0])                         # Set the Lower range value of blue in BGR
  upper_range = np.array([255,155,155])                     # Set the Upper range value of blue in BGR
  mask = cv2.inRange(im, lower_range, upper_range)     # Create a mask with range
  result = cv2.bitwise_and(im, im, mask = mask)   # Performing bitwise and operation with mask in img variable
  # cv2_imshow(result)

  return cv2.inRange(result, lower_range, upper_range)

# Try out
mask = getPlayersMask(img_out)
cv2_imshow(mask)

In [ ]:
def mask_team(mask, showResult = False):
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts_width = []
    for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      cnts_width.append(h)
    # print(cnts_width > 8)
    cnts_qualified = [ True if item > 4 else False for item in cnts_width ]
    # res = [ True if item in animal_list else False for item in your_list ]
    cnts_new = []
    for i in range(0,len(cnts)):
      # print(i)
      cnts_new.append(cnts[i][0])
    # cnts_new = [cnts_new[i] for i in (1,2,5)]
    cnts_new = list(compress(cnts_new, cnts_qualified))
    if showResult:
      cv2_imshow(img_out)
      cv2_imshow(mask)
    return cnts_new


cnts_new_array = mask_team(mask, True)
cnts_new_array

In [ ]:
import numpy as np
import pandas as pd

players_warrior = [7, 2, 5, 1, 10]
players_celtics = [8, 4, 3, 0, 6]

warrior_fg = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/warrior_fg.xlsx')
options_warrior = ['Curry', 'Thompson', 'Wiggins', 'Green', 'Looney']
warrior_court = warrior_fg[warrior_fg['Name'].isin(options_warrior)]
warrior_court.insert(1, 'id', players_warrior)

celtics_fg = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Celtics_fg.xlsx')
options_celtics = ['White', 'Brown', 'Tatum', 'Horford', 'Porzingis']
celtics_court = celtics_fg[celtics_fg['Name'].isin(options_celtics)]
celtics_court.insert(1, 'id', players_celtics)
warrior_court.head()


# minimap1

In [ ]:
def drawPlayersOnCourt(court_img, cnts, cnts_rim, players_warrior, players_celtics, j1, showResult=False):
  color_warrior = [255, 0, 0]
  color_celtics = [0, 0, 255]
  radius=10
  thickness = 70
  court = Polygon(src_pts)
  if j1 == 0:
    for i in players_warrior:
      # print(i)
      warrior_court.loc[warrior_court['id'] == i,'current_x'] = cnts[i][0][0]
      warrior_court.loc[warrior_court['id'] == i,'current_y'] = cnts[i][0][1]

      cv2.circle(court_img, (int(warrior_court.loc[warrior_court['id'] == i,'current_x']), int(warrior_court.loc[warrior_court['id'] == i,'current_y'])), 2, color_warrior, thickness)


      warrior_court.loc[warrior_court['id'] == i,'previous_x'] = warrior_court.loc[warrior_court['id'] == i,'current_x']
      warrior_court.loc[warrior_court['id'] == i,'previous_y'] = warrior_court.loc[warrior_court['id'] == i,'current_y']

    for i in players_celtics:

      celtics_court.loc[celtics_court['id'] == i,'current_x'] = cnts[i][0][0]
      celtics_court.loc[celtics_court['id'] == i,'current_y'] = cnts[i][0][1]
      cv2.circle(court_img, (int(celtics_court.loc[celtics_court['id'] == i,'current_x']), int(celtics_court.loc[celtics_court['id'] == i,'current_y'])), 2, color_celtics, thickness)

      celtics_court.loc[celtics_court['id'] == i,'previous_x'] = celtics_court.loc[celtics_court['id'] == i,'current_x']
      celtics_court.loc[celtics_court['id'] == i,'previous_y'] = celtics_court.loc[celtics_court['id'] == i,'current_y']
  else:
    cnts_newnew = []
    for i in range(len(cnts)):
      subarray1 = np.array([cnts[i][0][0], cnts[i][0][1]])
      cnts_newnew.append(subarray1)
    cnts_newnew_array = np.array(cnts_newnew)
    for i in players_warrior:
      previous_array = np.array([(warrior_court.loc[warrior_court['id'] == i, 'previous_x'].squeeze(), warrior_court.loc[warrior_court['id'] == i, 'previous_y'].squeeze())])
      dist = np.sqrt(((previous_array - cnts_newnew_array)**2).sum(axis = 1))
      if len(dist) == 0 or np.min(dist) > 180:
        warrior_court.loc[warrior_court['id'] == i,'current_x'] = warrior_court.loc[warrior_court['id'] == i,'previous_x']
        warrior_court.loc[warrior_court['id'] == i,'current_y'] = warrior_court.loc[warrior_court['id'] == i,'previous_y']
        cv2.circle(court_img, (int(warrior_court.loc[warrior_court['id'] == i,'current_x']), int(warrior_court.loc[warrior_court['id'] == i,'current_y'])), 2, color_warrior, thickness)

      else:
        index_nearest = np.argmin(dist)
        warrior_court.loc[warrior_court['id'] == i,'current_x'] = cnts_newnew_array[index_nearest][0]
        warrior_court.loc[warrior_court['id'] == i,'current_y'] = cnts_newnew_array[index_nearest][1]
        cv2.circle(court_img, (int(warrior_court.loc[warrior_court['id'] == i,'current_x']), int(warrior_court.loc[warrior_court['id'] == i,'current_y'])), 2, color_warrior, thickness)
        previous_array = np.array([(warrior_court.loc[warrior_court['id'] == i, 'previous_x'].squeeze(), warrior_court.loc[warrior_court['id'] == i, 'previous_y'].squeeze())])
        current_array = np.array([(warrior_court.loc[warrior_court['id'] == i, 'current_x'].squeeze(), warrior_court.loc[warrior_court['id'] == i, 'current_y'].squeeze())])
        # warrior_court.loc[warrior_court['id'] == i,'distance'] = warrior_court.loc[warrior_court['id'] == i,'distance'] + np.sqrt(((previous_array - current_array)**2).sum(axis = 1))
        old_distance = warrior_court.loc[warrior_court['id'] == i,'distance'] /30.638
        moving_distance = np.sqrt(((previous_array - current_array)**2).sum(axis = 1)) / 30.638
        warrior_court.loc[warrior_court['id'] == i,'distance'] = old_distance + moving_distance
        print('old:',old_distance, 'new:', moving_distance, 'all:', old_distance + moving_distance)
        row_to_remove = cnts_newnew_array[index_nearest]
        if np.any(cnts_newnew_array == row_to_remove):
          mask = np.any(cnts_newnew_array != row_to_remove, axis=1)
          cnts_newnew_array = cnts_newnew_array[mask]


      warrior_court.loc[warrior_court['id'] == i,'previous_x'] = warrior_court.loc[warrior_court['id'] == i,'current_x']
      warrior_court.loc[warrior_court['id'] == i,'previous_y'] = warrior_court.loc[warrior_court['id'] == i,'current_y']
      current_array = np.array([(warrior_court.loc[warrior_court['id'] == i, 'current_x'].squeeze(), warrior_court.loc[warrior_court['id'] == i, 'current_y'].squeeze())])
      dist_rim = np.sqrt(((current_array - cnts_rim[0])**2).sum(axis = 1))
      dist_rim = dist_rim / 30.638 - 2
      dist_rim = np.round(dist_rim)
      warrior_court.loc[warrior_court['id'] == i,'rim'] = dist_rim
      text = '{}:{}ft.'.format(warrior_court.loc[warrior_court['id'] == i,'Name'].values, dist_rim)
      cv2.putText(court_img, text, (int(warrior_court.loc[warrior_court['id'] == i,'current_x'])-120, int(warrior_court.loc[warrior_court['id'] == i,'current_y'])+70), cv2.FONT_HERSHEY_DUPLEX, 1.5, (0, 0, 0), 2)
    for i in players_celtics:
      previous_array = np.array([(celtics_court.loc[celtics_court['id'] == i, 'previous_x'].squeeze(), celtics_court.loc[celtics_court['id'] == i, 'previous_y'].squeeze())])
      dist = np.sqrt(((previous_array - cnts_newnew_array)**2).sum(axis = 1))
      print('previos:',previous_array)
      print('new:', cnts_newnew_array)
      print('dist:',dist)
      print('-------')
      if len(dist) == 0 or np.min(dist) > 180:
        celtics_court.loc[celtics_court['id'] == i,'current_x'] = celtics_court.loc[celtics_court['id'] == i,'previous_x']
        celtics_court.loc[celtics_court['id'] == i,'current_y'] = celtics_court.loc[celtics_court['id'] == i,'previous_y']
        cv2.circle(court_img, (int(celtics_court.loc[celtics_court['id'] == i,'current_x']), int(celtics_court.loc[celtics_court['id'] == i,'current_y'])), 2, color_celtics, thickness)

      else:
        index_nearest = np.argmin(dist)
        celtics_court.loc[celtics_court['id'] == i,'current_x'] = cnts_newnew_array[index_nearest][0]
        celtics_court.loc[celtics_court['id'] == i,'current_y'] = cnts_newnew_array[index_nearest][1]
        cv2.circle(court_img, (int(celtics_court.loc[celtics_court['id'] == i,'current_x']), int(celtics_court.loc[celtics_court['id'] == i,'current_y'])), 2, color_celtics, thickness)

        previous_array = np.array([(celtics_court.loc[celtics_court['id'] == i, 'previous_x'].squeeze(), celtics_court.loc[celtics_court['id'] == i, 'previous_y'].squeeze())])
        current_array = np.array([(celtics_court.loc[celtics_court['id'] == i, 'current_x'].squeeze(), celtics_court.loc[celtics_court['id'] == i, 'current_y'].squeeze())])
        celtics_court.loc[celtics_court['id'] == i,'distance'] = celtics_court.loc[celtics_court['id'] == i,'distance'] + np.sqrt(((previous_array - current_array)**2).sum(axis = 1))
        old_distance1 = celtics_court.loc[celtics_court['id'] == i,'distance']
        moving_distance1 = np.sqrt(((previous_array - current_array)**2).sum(axis = 1))
        celtics_court.loc[celtics_court['id'] == i,'distance'] = (old_distance1 + moving_distance1) / 30.638

        row_to_remove = cnts_newnew_array[index_nearest]
        if np.any(cnts_newnew_array == row_to_remove):
          mask = np.any(cnts_newnew_array != row_to_remove, axis=1)
          cnts_newnew_array = cnts_newnew_array[mask]

      celtics_court.loc[celtics_court['id'] == i,'previous_x'] = celtics_court.loc[celtics_court['id'] == i,'current_x']
      celtics_court.loc[celtics_court['id'] == i,'previous_y'] = celtics_court.loc[celtics_court['id'] == i,'current_y']
      current_array = np.array([(celtics_court.loc[celtics_court['id'] == i, 'current_x'].squeeze(), celtics_court.loc[celtics_court['id'] == i, 'current_y'].squeeze())])
      dist_rim = np.sqrt(((current_array - cnts_rim[0])**2).sum(axis = 1))
      celtics_court.loc[celtics_court['id'] == i,'rim'] = dist_rim
      dist_rim = np.round(dist_rim)
      text = '{}:{}'.format(celtics_court.loc[celtics_court['id'] == i,'Name'], dist_rim)
      # cv2.putText(court_img, text, (int(celtics_court.loc[celtics_court['id'] == i,'current_x'])-30, int(celtics_court.loc[celtics_court['id'] == i,'current_y'])+20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 0))


  if showResult:
    cv2_imshow(court_img)
  return court_img

# sample

In [ ]:
img_test1 = cv2.imread('frame3.jpg')
img_test2 = cv2.imread('frame5.jpg')
img_test3 = cv2.imread('frame6.jpg')
img_test4 = cv2.imread('frame7.jpg')
img_test5 = cv2.imread('frame8.jpg')
img_test6 = cv2.imread('frame9.jpg')
img_test7 = cv2.imread('frame10.jpg')
img_test8 = cv2.imread('frame11.jpg')
img_test9 = cv2.imread('frame12.jpg')
img_test10 = cv2.imread('frame13.jpg')
img_test11 = cv2.imread('frame14.jpg')
img_test12 = cv2.imread('frame15.jpg')
img_test13 = cv2.imread('frame16.jpg')
img_test14 = cv2.imread('frame17.jpg')
img_test15 = cv2.imread('frame18.jpg')
img_list = [img_test1, img_test2, img_test3, img_test4, img_test5, img_test6, img_test7, img_test8,
            img_test9, img_test10, img_test11, img_test12, img_test13, img_test14, img_test15]
j1 = 0
players_warrior = [7, 2, 5, 1, 10]
players_celtics = [8, 4, 3, 0, 6]
warrior_fg = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/warrior_fg.xlsx')
options_warrior = ['Curry', 'Thompson', 'Wiggins', 'Green', 'Looney']
warrior_court = warrior_fg[warrior_fg['Name'].isin(options_warrior)]
warrior_court.insert(1, 'id', players_warrior)

celtics_fg = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Celtics_fg.xlsx')
options_celtics = ['Russell', 'Reaves', 'James', 'Vanderbilt', 'Davis']
celtics_court = celtics_fg[celtics_fg['Name'].isin(options_celtics)]
celtics_court.insert(1, 'id', players_celtics)
celtics_court.head()
for img_tt in img_list:
  court_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/court3.jpg')
  players_output = predictor1(img_tt)
  instances = players_output["instances"]
  pred_boxes = instances.get("pred_boxes")
  pred_classes = instances.get("pred_classes")

  drawrim(img_tt, False)
  drawPlayers(img_tt, pred_boxes, pred_classes, False)
  img_out = homographyTransform(img_tt, court_img, False)
  Rimmask = getRimMask(img_out)
  cnts_rim = mask_rim(Rimmask, False)
  cnts_rim_array = np.array(cnts_rim)
  mask = getPlayersMask(img_out)
  cnts_new_array = mask_team(mask, False)
  print('--------')
  print(cnts_new_array)
  drawPlayersOnCourt(court_img, cnts_new_array, cnts_rim_array, players_warrior, players_celtics, j1, showResult=True)
  j1 = j1 + 1

# minimap_video

In [ ]:
def minimap_video(im, players_warrior, players_celtics, j1):
  court_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/court3.jpg')

  blue_color = (255,0,0)
  red_color = (0,0,255)

  players_output = predictor1(im)
  instances = players_output["instances"]
  pred_boxes = instances.get("pred_boxes")
  pred_classes = instances.get("pred_classes")
  drawrim(im, False)
  drawPlayers(im, pred_boxes, pred_classes, False)

  img_out = homographyTransform(im, court_img, False)
  Rimmask = getRimMask(img_out)
  cnts_rim = mask_rim(Rimmask, False)
  cnts_rim_array = np.array(cnts_rim)
  mask = getPlayersMask(img_out)
  cnts_new_array = mask_team(mask, False)
  print('-------------')
  print(cnts_new_array)
  result = drawPlayersOnCourt(court_img, cnts_new_array, cnts_rim_array,players_warrior, players_celtics, j1, showResult=False)

  return result

In [ ]:
import time
import progressbar
from time import sleep
from collections import deque
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import GenericMask
import imutils

vs = cv2.VideoCapture("/content/drive/MyDrive/Colab Notebooks/2k_top2.mp4")
totalFrames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))

grabbed = True
currentFrame = 0
start = time.time()
writer = None

bar = progressbar.ProgressBar(maxval=totalFrames, \
      widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

bar.start()

court_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/court3.jpg')
players_warrior = [7, 2, 5, 1, 10]
players_celtics = [8, 4, 3, 0, 6]
warrior_fg = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/warrior_fg.xlsx')
options_warrior = ['Curry', 'Thompson', 'Wiggins', 'Green', 'Looney']
warrior_court = warrior_fg[warrior_fg['Name'].isin(options_warrior)]
warrior_court.insert(1, 'id', players_warrior)

celtics_fg = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Celtics_fg.xlsx')
options_celtics = ['Russell', 'Reaves', 'James', 'Vanderbilt', 'Davis']
celtics_court = celtics_fg[celtics_fg['Name'].isin(options_celtics)]
celtics_court.insert(1, 'id', players_celtics)

j = 0
while grabbed:

  # read the next frame from the file
  (grabbed, frame) = vs.read()
  if j % 1 == 0:
    if writer is None:
      fourcc = cv2.VideoWriter_fourcc(*"mp4v")
      writer = cv2.VideoWriter("mini-map-output_2k_top_new180.mp4", fourcc, 59, (court_img.shape[1], court_img.shape[0]), True)

    if grabbed:
      court_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/court.png')
      result = minimap_video(frame, players_warrior, players_celtics, j)

      writer.write(result)

      currentFrame += 1
      bar.update(currentFrame)

    else:
        grabbed = False
    j = j + 1
  else:
    currentFrame += 1
    j = j + 1


writer.release()
vs.release()
bar.finish()

end = time.time()
elap = (end - start)
print("[INFO] process took {:.4f} seconds".format(elap))

print("Video created")

